# Fairness in the Multi-Secretary Problem

## Experiment 1: Pabulib Instances

In [ ]:
run_experiment_one()

## Experiment 2: Sushi and MovieLens Datasets.

### Sushi Dataset

In [ ]:
run_experiment_two_A()

### MovieLens Dataset

In [ ]:
run_experiment_two_B()

## Experiment 3: Sampled Datasets

### Impartial Culture

In [ ]:
run_experiment_three_A()

### Mallows Model

In [ ]:
run_experiment_three_B()

### Normalized Mallows Model

In [ ]:
run_experiment_three_C()

## Experiment 4: Polarized Instances

In [ ]:
run_experiment_four()

# -----------------------------------------------------------------------------------------

# Implementation

# Libraries and Logging

In [ ]:
from __future__ import annotations

import copy
import csv
import itertools
import logging
import math
import os
import random
import time
from collections import defaultdict
from pathlib import Path
from typing import Dict, Set, List, Optional, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

# Un-comment these lines if running in an environment where these are not installed
# !pip install colorama
# !pip install prefsampling

from colorama import Fore, Style
from prefsampling.ordinal import mallows, norm_mallows

# Configure Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(threadName)s] %(message)s',
    handlers=[
        logging.FileHandler("output.log"),
        logging.StreamHandler()
    ]
)

# Election Rules

In [ ]:
def greedy(utilities: np.ndarray, k: int, seed: int = None, exact_k: bool = True, return_free_riders: bool = False):
    """
    Greedy Budgeting Algorithm.
    
    Args:
        utilities: (n x m) utility matrix.
        k: Target committee size.
        seed: Random seed for tie-breaking/shuffling.
        exact_k: If True, ensures exactly k candidates are returned by filling with remaining candidates.
        return_free_riders: If True, returns a tuple (selected, free_riders).
    """
    n, m = utilities.shape
    # Each voter gets an equal budget share of k/n
    budget = np.ones(n) * (k / n)
    selected = []
    free_riders = []

    candidates = list(range(m))
    if seed is not None:
        rng = random.Random(seed)
        rng.shuffle(candidates)
    else:
        random.shuffle(candidates)
        
    remaining_candidates = candidates.copy()

    for c in candidates:
        if c in selected:
            continue

        remaining_candidates.remove(c)

        # Check if we need to fill the committee to reach size k immediately
        if exact_k:
            if len(selected) == k - len(remaining_candidates):
                selected.extend(remaining_candidates)
                free_riders = remaining_candidates.copy()
                break

        # Identify supporters who have utility and remaining budget
        supporters = [i for i in range(n) if utilities[i, c] > 0 and budget[i] > 0]
        if not supporters:
            continue

        supporters_budget = sum(budget[i] for i in supporters)
        if supporters_budget < 1.0 - 1e-10:
            continue  # Not enough budget to purchase candidate c (cost = 1.0)

        selected.append(c)

        # Distribute cost (1.0) equally among supporters, capped by their individual budget
        remaining_cost = 1.0
        contribs = {i: 0.0 for i in supporters}

        while remaining_cost > 1e-10:
            active = [i for i in supporters if budget[i] - contribs[i] > 1e-10]
            if not active:
                break

            share = remaining_cost / len(active)
            for i in active:
                pay = min(share, budget[i] - contribs[i])
                contribs[i] += pay
                remaining_cost -= pay

        # Deduct the calculated contributions from voter budgets
        for i in supporters:
            budget[i] -= contribs[i]

        if len(selected) == k:
            break

    if return_free_riders:
        return selected, free_riders
    else:
        return selected

In [ ]:
# --- Method of Equal Shares (Offline) Classes and Functions ---

class Voter:
    def __init__(self, id: str):
        self.id = id

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return self.id == other.id

    def __repr__(self):
        return f"v({self.id})"


class Candidate:
    def __init__(self, id: str):
        self.id = id
        self.cost = 1  # Unit cost assumption

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return self.id == other.id

    def __repr__(self):
        return f"c({self.id})"


class Election:
    def __init__(self, voters: Set[Voter] = None, profile: Dict[Candidate, Dict[Voter, int]] = None, budget: int = 0):
        self.voters = voters if voters else set()
        self.profile = profile if profile else {}  # Map: candidate -> voter -> utility
        self.budget = budget


def _utilitarian_greedy_internal(e: Election, W: set[Candidate]) -> set[Candidate]:
    """Internal helper to complete a committee using utilitarian greedy strategy."""
    costW = sum(c.cost for c in W)
    remaining = set(c for c in e.profile if c not in W)
    
    # Rank remaining candidates by bang-per-buck (total utility / cost)
    ranked = sorted(
        remaining,
        key=lambda c: (-sum(e.profile[c].values()) / c.cost, c.id)
    )
    
    for c in ranked:
        if costW + c.cost <= e.budget:
            W.add(c)
            costW += c.cost
    return W


def utilitarian_greedy(e: Election) -> set[Candidate]:
    return _utilitarian_greedy_internal(e, set())


def safe_division(x, y):
    """Safely divides x by y, handling small denominators."""
    if abs(y) < 1e-10:
        return float('inf')
    else:
        return x / y


def _mes_internal(e: Election, real_budget: int = 0) -> Tuple[Dict[Voter, float], Set[Candidate]]:
    """
    Internal implementation of the Method of Equal Shares (MES).
    Returns:
        endow: Remaining budget per voter.
        W: Set of winning candidates.
    """
    W = set()
    costW = 0
    remaining = set(c for c in e.profile)
    endow = {i: 1.0 * e.budget / len(e.voters) for i in e.voters}
    
    # rho represents the cost-per-utility unit required to elect a candidate
    rho = {c: safe_division(c.cost, sum(e.profile[c].values())) for c in e.profile}

    while True:
        next_candidate = None
        lowest_rho = float("inf")
        remaining_sorted = sorted(remaining, key=lambda c: (rho[c], c.id))

        for c in remaining_sorted:
            if rho[c] > lowest_rho + 1e-10:
                break
            if e.budget > 1e6: # Safety break for extremely large budgets
                break
                
            # Check if supporters have enough current budget to afford the candidate
            if sum(endow[i] for i in e.profile[c] if e.profile[c][i] > 0) + 1e-8 >= c.cost:
                supporters_sorted = sorted(
                    [i for i in e.profile[c]],
                    key=lambda i: safe_division(endow[i], e.profile[c][i])
                )
                price = c.cost
                util = sum(e.profile[c].values())

                # Water-filling algorithm to find the clearing price (rho)
                for i in supporters_sorted:
                    if endow[i] * util >= price * e.profile[c][i]:
                        break
                    price -= endow[i]
                    util -= e.profile[c][i]

                if price > 1e-5:
                    rho[c] = price / util
                else:
                    rho[c] = safe_division(endow[supporters_sorted[-1]], e.profile[c][supporters_sorted[-1]])

                # Select candidate with the minimum rho (lexicographic tie-breaking)
                if (rho[c] < lowest_rho or
                    (abs(rho[c] - lowest_rho) < 1e-10 and (next_candidate is None or c.id < next_candidate.id))):
                    next_candidate = c
                    lowest_rho = rho[c]

        if next_candidate is None:
            break
        else:
            W.add(next_candidate)
            costW += next_candidate.cost
            remaining.remove(next_candidate)
            # Deduct budget from supporters
            for i in e.profile[next_candidate]:
                endow[i] -= min(endow[i], lowest_rho * e.profile[next_candidate][i])
            
            # Optimization for 'increase-budget' completions
            if real_budget: 
                if costW > real_budget:
                    return None
    return endow, W


def _is_exhaustive(e: Election, W: set[Candidate]) -> bool:
    """Checks if budget is exhausted such that no other candidate can be afforded."""
    costW = sum(c.cost for c in W)
    minRemainingCost = min([c.cost for c in e.profile if c not in W], default=math.inf)
    return costW + minRemainingCost > e.budget


def equal_shares(e: Election, completion: str = None) -> set[Candidate]:
    """
    Executes Method of Equal Shares with optional completion strategies.
    
    strategies:
        - 'binsearch': Binary search for optimal budget multiplier.
        - 'utilitarian_greedy': Fills remainder with utilitarian greedy.
        - 'add1': Incrementally increases budget.
        - None: Standard MES.
    """
    endow, W = _mes_internal(e)
    if completion is None:
        return W
        
    if completion == 'binsearch':
        initial_budget = e.budget
        # Double budget until exhaustive
        while not _is_exhaustive(e, W):
            if e.budget > 1e6: break
            b_low = e.budget
            e.budget *= 2
            res_nxt = _mes_internal(e, real_budget=initial_budget)
            if res_nxt is None: break
            _, W = res_nxt
            
        # Binary search between low and high
        b_high = e.budget
        while not _is_exhaustive(e, W) and b_high - b_low >= 1:
            e.budget = (b_high + b_low) / 2.0
            res_med = _mes_internal(e, real_budget=initial_budget)
            if res_med is None:
                b_high = e.budget
            else:
                b_low = e.budget
                _, W = res_med
        e.budget = initial_budget
        return W

    if completion == 'utilitarian_greedy':
        return _utilitarian_greedy_internal(e, W)

    if completion == 'add1':
        initial_budget = e.budget
        while not _is_exhaustive(e, W):
            if e.budget > 1e6: break
            e.budget *= 1.01
            res_nxt = _mes_internal(e, real_budget=initial_budget)
            if res_nxt is None: break
            _, W = res_nxt
        e.budget = initial_budget
        return W
    
    if completion == 'add1_utilitarian':
        initial_budget = e.budget
        while not _is_exhaustive(e, W):
            if e.budget > 1e6: break
            e.budget *= 1.01
            res_nxt = _mes_internal(e, real_budget=initial_budget)
            if res_nxt is None: break
            _, W = res_nxt
        e.budget = initial_budget
        return _utilitarian_greedy_internal(e, W)

    assert False, f"Invalid value of parameter completion: {completion}"

In [ ]:
def online_mes(utils: np.ndarray, k: int, seed: int = None, MEScompletion: str = None, exact_k: bool = True):
    """
    Online Variant of the Method of Equal Shares.
    Uses a sample phase (1/e of candidates) to estimate prices/budget, 
    then makes irrevocable decisions for the remaining stream.
    """
    n, m = utils.shape
    # Sample size threshold (1/e rule for secretary problems)
    t = math.floor(m / math.e)

    full_election = convert_input_to_election(utils, k)
    all_candidates = list(full_election.profile.keys())
    candidate_ids = list(range(len(all_candidates)))

    if seed is not None:
        rng = random.Random(seed)
        rng.shuffle(candidate_ids)
    else:
        random.shuffle(candidate_ids)
        
    initial_ids = candidate_ids[:t]
    remaining_ids = candidate_ids[t:]

    # Run MES on the sample to establish reference
    initial_profile = {
        all_candidates[i]: full_election.profile[all_candidates[i]]
        for i in initial_ids
    }
    initial_election = Election(voters=full_election.voters, budget=k)
    initial_election.profile = initial_profile
    initial_selection = equal_shares(initial_election, MEScompletion)

    C_R = [all_candidates.index(c) for c in initial_selection]
    C_R_r = C_R.copy()
    selected = []

    # Process the stream
    for i, c_id in enumerate(remaining_ids):
        remaining_count = len(remaining_ids) - i
        
        # Force selection if we are running out of candidates to fill k
        if exact_k:
            if len(selected) == k - remaining_count:
                selected.extend(remaining_ids[i:])
                break

        candidate_set_ids = C_R_r + [c_id]
        candidate_set = [all_candidates[i] for i in candidate_set_ids]
        sub_profile = {c: full_election.profile[c] for c in candidate_set}

        sub_election = Election(voters=full_election.voters, budget=k)
        sub_election.profile = sub_profile
        new_selection = equal_shares(sub_election, MEScompletion)
        new_selection_ids = [all_candidates.index(c) for c in new_selection]

        # If the current candidate is selected in the hypothetical offline run including history, select it
        if c_id in new_selection_ids:
            replaced = set(C_R_r) - set(new_selection_ids)
            if any(r in C_R for r in replaced):
                selected.append(c_id)

        C_R_r = new_selection_ids

    # Final fill if we didn't reach k (using sample or remaining)
    if exact_k:
        if len(selected) < k:
            remaining_initial = [c for c in initial_ids if c not in selected]
            needed = k - len(selected)
            selected.extend(remaining_initial[:needed])

            if len(selected) < k:
                remaining_all = [c for c in candidate_ids if c not in selected]
                needed = k - len(selected)
                selected.extend(remaining_all[:needed])
        return selected[:k]
    return selected


In [ ]:
def bounded_overspending(e: Election, real_budget: int = 0) -> Set[Candidate]:
    """
    Offline Bounded Overspending (BOS) implementation.
    Calculates a 'ratio' for every candidate to determine eligibility.
    """
    W = set()
    costW = 0
    remaining = set(c for c in e.profile)
    endow = {i: 1.0 * safe_division(e.budget, len(e.voters)) for i in e.voters}
    ratio = {c: -1.0 for c in e.profile}
    
    while True:
        next_candidate = None
        lowest_ratio = float("inf")
        remaining_sorted = sorted(remaining, key=lambda c: (ratio[c], c.id))
        best_util = 0
        
        for c in remaining_sorted:
            if ratio[c] >= lowest_ratio:
                break
            if costW + c.cost <= e.budget:
                supporters_sorted = sorted([i for i in e.profile[c]], key=lambda i: safe_division(endow[i], e.profile[c][i]))
                util = sum(e.profile[c].values())
                money_used = 0
                last_rho = 0
                new_ratio = float("inf")
                
                # Determine the specific ratio for this candidate
                for i in supporters_sorted:
                    denominator = e.profile[c][i]
                    if denominator == 0 or not np.isfinite(denominator):
                        continue 

                    numerator = endow[i]
                    if not np.isfinite(numerator):
                        continue

                    util_safe = util if np.isfinite(util) else 0
                    alpha = min(1.0, safe_division((money_used + util_safe * safe_division(numerator, denominator)), c.cost))
                    
                    if alpha > 1e-8:
                        rho = safe_division(((alpha * c.cost) - money_used), (alpha * util))
                        if rho < last_rho:
                            break
                        if rho / alpha < new_ratio :
                            new_ratio = safe_division(rho, alpha)
                            new_rho = rho
                            
                    util -= e.profile[c][i]
                    money_used += endow[i]
                    last_rho = safe_division(endow[i], e.profile[c][i])
                
                ratio[c] = new_ratio
                if ratio[c] < lowest_ratio:
                    lowest_ratio = ratio[c]
                    lowest_rho = new_rho
                    next_candidate = c
                    best_util = sum([e.profile[c][i] for i in e.profile[c]])
                elif ratio[c] == lowest_ratio:
                    util = sum([e.profile[c][i] for i in e.profile[c]])
                    if util > best_util:
                        next_candidate = c
                        best_util = util
                        
        if next_candidate is None:
            break
        else:
            W.add(next_candidate)
            costW += next_candidate.cost
            remaining.remove(next_candidate)
            for i in e.profile[next_candidate]:
                endow[i] -= min(endow[i], lowest_rho * e.profile[next_candidate][i])
            if real_budget: 
                if costW > real_budget:
                    return None
    return W

In [ ]:
def online_bos(utils: np.ndarray, k: int, seed: int = None, exact_k: bool = True):
    """
    Online Variant of BOS (Bounded Overspending).
    Uses the sample phase technique to establish initial ratios.
    """
    n, m = utils.shape
    t = math.floor(m / math.e)

    full_election = convert_input_to_election(utils, k)
    all_candidates = list(full_election.profile.keys())
    candidate_ids = list(range(len(all_candidates)))

    if seed is not None:
        rng = random.Random(seed)
        rng.shuffle(candidate_ids)
    else:
        random.shuffle(candidate_ids)
        
    initial_ids = candidate_ids[:t]
    remaining_ids = candidate_ids[t:]
    
    initial_profile = {
        all_candidates[i]: full_election.profile[all_candidates[i]]
        for i in initial_ids
    }
    initial_election = Election(voters=full_election.voters, budget=k)
    initial_election.profile = initial_profile
    initial_selection = bounded_overspending(initial_election)
    
    C_R = [all_candidates.index(c) for c in initial_selection]
    C_R_r = C_R.copy()
    selected = []

    for i, c_id in enumerate(remaining_ids):
        remaining_count = len(remaining_ids) - i
        if exact_k:
            if len(selected) == k - remaining_count:
                selected.extend(remaining_ids[i:])
                break

        candidate_set_ids = C_R_r + [c_id]
        candidate_set = [all_candidates[i] for i in candidate_set_ids]
        sub_profile = {c: full_election.profile[c] for c in candidate_set}

        sub_election = Election(voters=full_election.voters, budget=k)
        sub_election.profile = sub_profile
        new_selection = bounded_overspending(sub_election)
        new_selection_ids = [all_candidates.index(c) for c in new_selection]
        
        if c_id in new_selection_ids:
            selected.append(c_id)

        C_R_r = new_selection_ids
        
    if exact_k:
        if len(selected) < k:
            remaining_initial = [c for c in initial_ids if c not in selected]
            selected.extend(remaining_initial[:k - len(selected)])

            if len(selected) < k:
                remaining_all = [c for c in candidate_ids if c not in selected]
                selected.extend(remaining_all[:k - len(selected)])

    return selected[:k]

In [ ]:
def utility_function_approval(voter, selected_projects, votes):
    return sum(1 for project in votes[voter] if project in selected_projects)


def utility_function_additive(voter, selected_projects, utilities, full_to_real_index):
    total_utility = 0
    voter_idx = int(voter)
    voter_utilities = utilities[voter_idx]

    for full_proj_idx in selected_projects:
        real_proj_idx = full_to_real_index.get(full_proj_idx)
        if real_proj_idx is None:
            # dummy project, contributes 0
            continue
        total_utility += voter_utilities.get(real_proj_idx, 0)

    return total_utility


def utility_function(voter, selected_projects, votes, vote_type, full_to_real_index=None):
    match vote_type:
        case "Approval":
            return utility_function_approval(voter, selected_projects, votes)
        case "Additive":
            if full_to_real_index is None:
                raise ValueError("full_to_real_index must be provided for Additive vote type")
            return utility_function_additive(voter, selected_projects, votes, full_to_real_index)
        case _:
            logging.critical("Requested Vote Type is Not Supported!")


def social_welfare(selected_projects, voters, votes, vote_type, full_to_real_index=None):
    return sum(math.log(1 + utility_function(voter, selected_projects, votes, vote_type, full_to_real_index)) for voter in voters)


def is_dummy_project(project):
    return isinstance(project, str) and project.startswith("__dummy__")


def insert_dummy_projects_randomly(projects, k, seed_dum=None):
    """Inserts dummy projects to ensure total candidates is a multiple of k."""
    n = len(projects)
    remainder = n % k
    if remainder == 0:
        return projects

    needed = k - remainder
    dummy_projects = [f"__dummy__{i}" for i in range(needed)]

    rng = random.Random(seed_dum)
    insert_positions = sorted(rng.sample(range(n + 1), needed))

    result = []
    dummy_idx = 0
    proj_idx = 0
    for i in range(n + needed):
        if dummy_idx < needed and i == insert_positions[dummy_idx] + dummy_idx:
            result.append(dummy_projects[dummy_idx])
            dummy_idx += 1
        else:
            result.append(projects[proj_idx])
            proj_idx += 1

    return result


def monotone_submodular_secretary(k, projects, voters, votes, vote_type, seed=None):
    """
    Implementation of the Monotone Submodular Secretary algorithm (Online Nash).
    Divides candidates into k phases and selects at most 1 per phase.
    """
    real_projects = [p for p in projects if not is_dummy_project(p)]
    indexed_real_projects = list(enumerate(real_projects))

    if seed is not None:
        rng = random.Random(seed)
        rng.shuffle(indexed_real_projects)
    else:
        random.shuffle(indexed_real_projects)

    shuffled_real_projects = [p for _, p in indexed_real_projects]

    # Pad with dummy projects if necessary
    if len(shuffled_real_projects) % k != 0:
        projects_with_dummies = insert_dummy_projects_randomly(
            shuffled_real_projects, k, seed_dum=(seed + 1 if seed is not None else None)
        )
    else:
        projects_with_dummies = shuffled_real_projects

    real_proj_mapping = {p: (idx, p) for idx, p in indexed_real_projects}

    project_to_full_index = {}
    for i, proj in enumerate(projects_with_dummies):
        if not is_dummy_project(proj):
            project_to_full_index[proj] = i

    full_to_real_index = {}
    for i, proj in enumerate(projects_with_dummies):
        if is_dummy_project(proj):
            full_to_real_index[i] = None
        else:
            full_to_real_index[i] = real_proj_mapping[proj][0]

    final_indexed_projects = []
    for proj in projects_with_dummies:
        if is_dummy_project(proj):
            final_indexed_projects.append((None, proj))
        else:
            final_indexed_projects.append(real_proj_mapping[proj])

    selected_projects = []
    elements_per_phase = len(final_indexed_projects) // k

    # Phase-based selection
    for phase in range(1, k + 1):
        start_index = (phase - 1) * elements_per_phase
        upper_index = start_index + int(elements_per_phase / math.e)
        phase_end_index = phase * elements_per_phase

        alpha = float("-inf")

        # Observation sub-phase (finding threshold alpha)
        for index in range(start_index, min(upper_index, len(final_indexed_projects))):
            _, project = final_indexed_projects[index]
            candidate_set = [p for _, p in selected_projects] + [project]
            alpha = max(alpha, social_welfare(candidate_set, voters, votes, vote_type, full_to_real_index))

        current_value = social_welfare([p for _, p in selected_projects], voters, votes, vote_type, full_to_real_index)
        alpha = max(alpha, current_value)

        # Selection sub-phase
        project_selected = None
        for index in range(upper_index, min(phase_end_index, len(final_indexed_projects))):
            idx, project = final_indexed_projects[index]
            if is_dummy_project(project):
                continue
            candidate_set = [p for _, p in selected_projects] + [project]
            value = social_welfare(candidate_set, voters, votes, vote_type, full_to_real_index)
            if value >= alpha and project_selected is None:
                project_selected = (idx, project)

        # If nothing selected, force selection from the end of phase (if possible)
        if project_selected is None:
            for index in reversed(range(start_index, min(phase_end_index, len(final_indexed_projects)))):
                idx, project = final_indexed_projects[index]
                if not is_dummy_project(project):
                    project_selected = (idx, project)
                    break

        if project_selected:
            selected_projects.append(project_selected)

    result_indices = []
    for idx, proj in selected_projects:
        if idx is not None:
            full_idx = project_to_full_index[proj]
            real_idx = full_to_real_index[full_idx]
            if real_idx is not None:
                result_indices.append(real_idx)
    return result_indices

# Data Conversion and Functions for Comparison of Outputs

In [ ]:
def convert_pabulibdata(projects, votes, output_type):
    """Routes Pabulib data conversion based on algorithm requirement."""
    match output_type:
        case "SubmodularSecretary":
            project_list = list(projects.keys())
            voter_list = list(votes.keys())
            votes_by_voter = {voter: votes[voter]['vote'] for voter in voter_list}
            return project_list, voter_list, votes_by_voter
        case "MES" | "Greedy" | "BOS":
            project_list = list(projects.keys())
            voter_list = list(votes.keys())
            votes_by_voter = {voter: votes[voter]['vote'] for voter in voter_list}
            utilities = []
            for voter_id in voter_list:
                voter_utilities = [1 if project in votes_by_voter[voter_id] else 0 for project in project_list]
                utilities.append(voter_utilities)
            return np.array(utilities)
        case _:
            logging.critical("Requested Convert Type is Not Supported!")

def convert_matrix_utils(matrix_utils, output_type):
    match output_type:
        case "SubmodularSecretary":
            n, m = len(matrix_utils), len(matrix_utils[0])
            project_list = list(range(m))
            voter_list = list(range(n))
            votes_by_voter = [
                {j: matrix_utils[i][j] for j in range(m) if matrix_utils[i][j] > 0}
                for i in range(n)
            ]
            return project_list, voter_list, votes_by_voter
        case "MES" | "Greedy" | "BOS":
            return matrix_utils
        case _:
            logging.critical("Requested Convert Type is Not Supported!")


def read_csv_file(filepath):
    """Reads Pabulib format CSV files."""
    meta, projects, votes = {}, {}, {}
    project_approval_count = {}

    with open(filepath, 'r', newline='', encoding="utf-8") as csvfile:
        section = ""
        reader = csv.reader(csvfile, delimiter=';')

        for row in reader:
            if row[0].strip().lower() in ["meta", "projects", "votes"]:
                section = row[0].strip().lower()
                header = next(reader)
            elif section == "meta":
                meta[row[0]] = row[1].strip()
            elif section == "projects":
                projects[row[0]] = {key.strip(): row[it + 1].strip() for it, key in enumerate(header[1:])}
                project_approval_count[row[0]] = 0
            elif section == "votes":
                votes[row[0]] = {key.strip(): row[it + 1].strip() for it, key in enumerate(header[1:])}

    for voter_data in votes.values():
        for project in voter_data.get('vote', '').split(','):
            project_approval_count[project] += 1

    return meta, projects, votes, project_approval_count


def convert_input_to_election(input_matrix: np.ndarray, k: int) -> Election:
    voters = [Voter(str(i)) for i in range(input_matrix.shape[0])]
    candidates = [Candidate(f"c{i}") for i in range(input_matrix.shape[1])]
    election = Election(voters=set(voters), budget=k)
    
    profile = {}
    for j in range(input_matrix.shape[1]):
        candidate = candidates[j]
        profile[candidate] = {}
        for i in range(input_matrix.shape[0]):
            profile[candidate][voters[i]] = input_matrix[i, j]

    election.profile = profile
    return election


def average_satisfaction(utils, winners):
    """Calculate the average satisfaction of voters for a given winning committee."""
    n, m = utils.shape
    if not isinstance(winners, list):
        winners = list(winners)
        
    valid_winners = [w for w in winners if isinstance(w, int) and 0 <= w < m]
    if not winners:
        return 0

    satisfaction = np.sum(utils[:, winners], axis=1)
    all_satisfaction = satisfaction[satisfaction >= 0]
    return np.mean(all_satisfaction)


def exclusion_ratio(utils, winners):
    """Calculate the number of voters with zero satisfaction."""
    n, m = utils.shape
    if not isinstance(winners, list):
        winners = list(winners)
        
    satisfaction = np.sum(utils[:, winners], axis=1)
    zero_satisfaction_count = np.sum(satisfaction == 0)
    return zero_satisfaction_count


def ejr_plus_violations(utils, winners, k, candidate_costs=None):
    """
    Calculates EJR+ violations (for approval instances).
    Returns: (count of violations, average shortfall)
    """
    n, m = utils.shape
    if candidate_costs is None:
        candidate_costs = np.ones(m)

    winners_set = set(winners)
    satisfaction = np.sum(utils[:, list(winners_set)], axis=1)
    sorted_voters = np.argsort(satisfaction)

    violations = 0
    total_shortfall = 0.0

    for j in range(m):
        if j in winners_set:
            continue

        supporters = np.where(utils[:, j] > 0)[0]
        if len(supporters) == 0:
            continue

        threshold = (len(supporters) / n) * k

        for i in sorted_voters:
            if i in supporters:
                if satisfaction[i] < threshold:
                    violations += 1
                    total_shortfall += (threshold - satisfaction[i])
                    break

    avg_shortfall = total_shortfall / violations if violations > 0 else 0.0
    return violations, avg_shortfall


def satisfaction_gini(utils, winners):
    """Compute the Gini coefficient of voter satisfaction."""
    if len(winners) == 0:
        return 0.0

    satisfaction = np.sum(utils[:, winners], axis=1)
    satisfaction_sorted = np.sort(satisfaction)
    n = len(satisfaction_sorted)
    index = np.arange(1, n + 1)

    total = np.sum(satisfaction_sorted)
    if total == 0:
        return 0.0

    gini = np.sum((2 * index - n - 1) * satisfaction_sorted) / (n * total)
    return gini


def percentile_utility(utils, winners, percentile=25):
    satisfaction = np.sum(utils[:, winners], axis=1)
    return np.percentile(satisfaction, percentile)


def compare_algorithms(k_values, seeds, n, m, positive_fraction, max_utility, utils=None):
    """
    Runs comparison suite for generated/provided utilities across multiple algorithms.
    """
    results = {}

    for k in k_values:
        results[k] = {
            alg: {
                "Avg Satisfaction": [], "Exclusion Ratio": [], "Gini": [], 
                "10th Percentile": [], "15th Percentile": [], "25th Percentile": []
            } for alg in ["Greedy Budgeting", "Online MES", "Online BOS", "Offline MES", "Online Nash Rule"]
        }

        for seed in seeds:
            if utils is None:
                utils = create_random_utilities(n, m, positive_fraction, max_utility, seed)

            election = convert_input_to_election(utils, k)

            # Offline MES
            result_offline_mes_temp = equal_shares(election, completion='utilitarian_greedy')
            result_offline_mes = sorted([int(''.join(filter(str.isdigit, str(c)))) for c in result_offline_mes_temp])

            # Online Algorithms
            selected_greedy = greedy(utils, k, seed)
            selected_online_mes_greedycompletion = online_mes(utils, k, seed, 'utilitarian_greedy')
            selected_online_bos = online_bos(utils, k, seed)

            # Submodular Secretary
            projects_list, voters_list, votes = convert_matrix_utils(utils, "SubmodularSecretary")
            selected_submod = monotone_submodular_secretary(k, projects_list, voters_list, votes, "Additive", seed)

            # Compute Metrics
            for algo_name, selected in [
                ("Greedy Budgeting", selected_greedy),
                ("Online BOS", selected_online_bos),
                ("Online MES", selected_online_mes_greedycompletion),
                ("Offline MES", result_offline_mes),
                ("Online Nash Rule", selected_submod)
            ]:
                avg_sat = average_satisfaction(utils, selected)
                excl_ratio = exclusion_ratio(utils, selected)
                gini = satisfaction_gini(utils, selected)
                p10 = percentile_utility(utils, selected, percentile=10)
                p15 = percentile_utility(utils, selected, percentile=15)
                p25 = percentile_utility(utils, selected, percentile=25)

                for metric, val in zip(
                    ["Avg Satisfaction", "Exclusion Ratio", "Gini", "10th Percentile", "15th Percentile", "25th Percentile"],
                    [avg_sat, excl_ratio, gini, p10, p15, p25]
                ):
                    results[k][algo_name][metric].append(val)
    return results

# Visual configurations
color_marker = {
    "Online BOS": ("#2ca02c", "o"),
    "Offline MES": ("#000000", "x"),
    "Greedy Budgeting": ("#1f77b4", "s"),
    "Online MES": ("#ff7f0e", "^"),
    "Online Nash Rule": ("#d62728", "+")
}

def plot_metric(metric):
    """Generates absolute metric plots."""
    output_dir = "./plots"
    os.makedirs(output_dir, exist_ok=True)

    plt.rcParams.update({
        'font.family': 'Times New Roman',
        'axes.titlesize': 20,
        'axes.labelsize': 18,
        'xtick.labelsize': 16,
        'ytick.labelsize': 16,
        'legend.fontsize': 16,
    })

    var_labels = {
        "n": "Number of Voters",
        "m": "Number of Candidates",
        "k": "Committee Size",
        "p": "Approval Probability"
    }

    for var in ["n", "m", "k", "p"]:
        plt.figure(figsize=(12, 6))
        plt.title(f"{metric} vs {var_labels[var]}")

        for alg in color_marker:
            color, marker = color_marker[alg]
            clean_label = alg.replace(" with utilitarian completion", "")
            x_vals = sorted(set(entry[var] for entry in all_results))
            y_vals, y_errs = [], []

            for v in x_vals:
                entries = [entry for entry in all_results if entry[var] == v and entry["algorithm"] == alg]
                if not entries:
                    continue
                y = np.mean([entry[metric] for entry in entries])
                err = np.mean([entry[metric + " std"] for entry in entries])
                y_vals.append(y)
                y_errs.append(err)

            if y_vals:
                plt.errorbar(x_vals, y_vals, yerr=y_errs, label=clean_label,
                             color=color, marker=marker, linestyle='-')

        plt.xlabel(var_labels[var])
        plt.ylabel(metric)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        
        # Save logic
        check_save = True
        check_save_counter = 0
        while check_save:
            plot_filename = f"{output_dir}/{metric.replace(' ', '_')}_vs_{var}_{check_save_counter}.png"
            check_save_counter += 1
            if not os.path.exists(plot_filename):
                plt.savefig(plot_filename)
                check_save = False
                break
        plt.show()
        plt.close()

def plot_relative_to_mes(metric):
    """Generates plots showing difference relative to Offline MES."""
    output_dir = "./plots"
    os.makedirs(output_dir, exist_ok=True)
    
    var_labels = {
        "n": "Number of Voters",
        "m": "Number of Candidates",
        "k": "Committee Size",
        "p": "Approval Probability"
    }

    for var in ["n", "k", "m", "p"]:
        plt.figure(figsize=(12, 6))
        plt.title(f"Absolute Relative Difference to Offline MES: {metric} vs {var_labels[var]}")
        x_vals = sorted(set(entry[var] for entry in all_results))

        for alg in color_marker:
            if alg == "Offline MES": continue
            color, marker = color_marker[alg]
            clean_label = alg.replace(" with utilitarian completion", "")
            rel_diffs, filtered_x = [], []

            for v in x_vals:
                entries_alg = [entry for entry in all_results if entry[var] == v and entry["algorithm"] == alg]
                entries_mes = [entry for entry in all_results if entry[var] == v and entry["algorithm"] == "Offline MES"]
                if not entries_alg or not entries_mes: continue

                y_alg = np.mean([entry[metric] for entry in entries_alg])
                y_mes = np.mean([entry[metric] for entry in entries_mes])

                if y_mes == 0:
                    rel_diff = abs(y_alg - y_mes)
                else:
                    rel_diff = abs(y_alg - y_mes) / y_mes

                rel_diffs.append(rel_diff)
                filtered_x.append(v)

            if rel_diffs:
                plt.plot(filtered_x, rel_diffs, label=clean_label, color=color, marker=marker, linestyle='-')

        plt.xlabel(var_labels[var])
        plt.ylabel("Abs. Relative Difference")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

# Experiment 1

In [ ]:
def classify_file(m):
    if m < 10: return 'Small'
    elif m >= 30: return 'Large'
    else: return 'Medium'

def run_experiment_one():
    """Reads Pabulib data and plots EJR+ violations."""
    data_dir = "sample_data/PabulibData"
    seeds = list(range(1, 6))
    num_runs = 1
    rule_names = ['Greedy Budgeting', 'Online MES', 'Online BOS', 'Online Nash Rule']
    
    violations_data = defaultdict(lambda: defaultdict(list))
    shortfall_data = defaultdict(lambda: defaultdict(list))
    grouped_violations = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    
    file_number_counter = -1
    fractions = [20, 15, 10, 7, 4, 2]
    
    for filepath in Path(data_dir).iterdir():
        if filepath.is_file():
            file_number_counter += 1
            if file_number_counter % 50 == 0:
                logging.info(f"Processing File: {filepath.stem}")
            
            try:
                meta, projects, votes, _ = read_csv_file(filepath)
                m = len(projects)
                size_class = classify_file(m)
                
                project_list, voter_list, votes_by_voter = convert_pabulibdata(projects, votes, "SubmodularSecretary")
                utils = convert_pabulibdata(projects, votes, "Greedy")

                for fraction in fractions:
                    m_fraction = m // fraction
                    if m_fraction <= 0: continue

                    for seed in seeds:
                        for run in range(num_runs):
                            selected = {}
                            # Run Algorithms
                            selected['Greedy Budgeting'] = greedy(utils, m_fraction, seed + run)
                            selected['Online MES'] = online_mes(utils, m_fraction, seed + run, 'utilitarian_greedy')
                            selected['Online BOS'] = online_bos(utils, m_fraction, seed + run)
                            selected['Online Nash Rule'] = monotone_submodular_secretary(
                                m_fraction, projects=project_list, voters=voter_list,
                                votes=votes_by_voter, vote_type="Approval", seed=seed + run
                            )

                            # Check Violations
                            for rule in rule_names:
                                winners = [int(w) for w in selected[rule] if int(w) < m]
                                raw_violations, avg_shortfall = ejr_plus_violations(utils, winners, m_fraction)
                                ratio_violations = raw_violations / len(votes)
                                violations_data[fraction][rule].append(ratio_violations)
                                shortfall_data[fraction][rule].append(avg_shortfall)
                                grouped_violations[size_class][fraction][rule].append(ratio_violations)
                                grouped_violations["All"][fraction][rule].append(ratio_violations)

            except Exception as e:
                logging.error(f"Error loading file {filepath.name}: {e}")

    # === Plot: Bar chart ===
    all_fractions = sorted(set(violations_data.keys()), reverse=True)
    fig, ax = plt.subplots(figsize=(14, 7))
    bar_width = 0.15
    x = np.arange(len(all_fractions))
    colors = {
        'Greedy Budgeting': '#1f77b4',
        'Online MES': '#ff7f0e',
        'Online BOS': '#2ca02c',
        'Online Nash Rule': '#d62728'
    }

    for i, rule in enumerate(rule_names):
        avg_heights = [np.mean(violations_data[fraction][rule]) if violations_data[fraction][rule] else 0 for fraction in all_fractions]
        max_heights = [np.max(violations_data[fraction][rule]) if violations_data[fraction][rule] else 0 for fraction in all_fractions]
        positions = x + (i - len(rule_names) / 2) * bar_width + bar_width / 2
        ax.bar(positions, avg_heights, width=bar_width, label=f"{rule} (avg)", color=colors[rule], alpha=0.7)
        diff = [max_h - avg_h if max_h > avg_h else 0 for max_h, avg_h in zip(max_heights, avg_heights)]
        ax.bar(positions, diff, width=bar_width, bottom=avg_heights, label=f"{rule} (max)", color=colors[rule], alpha=0.3, hatch='//')

    ax.set_xticks(x)
    ax.set_xticklabels([f'm/{fraction}' for fraction in all_fractions])
    ax.set_ylabel("EJR+ Violations (Voter Proportion in 0–1 Scale)")
    ax.set_title("Average and Max EJR+ Violations by Rule and Committee Size")
    ax.legend(loc='upper right', fontsize='small')
    plt.tight_layout()
    plt.show()

    # === Tables: Average, Max, and Confidence Intervals ===
    def build_violation_table(stat_fn=np.mean):
        df = pd.DataFrame(index=["Small", "Medium", "Large", "All"])
        for rule in rule_names:
            for fraction in fractions:
                col_name = f"{rule} m/{fraction}"
                for size_class in df.index:
                    values = grouped_violations[size_class][fraction][rule]
                    df.loc[size_class, col_name] = stat_fn(values) if values else 0
        return df.astype(float)

    def build_shortfall_table(stat_fn=np.mean):
        df = pd.DataFrame(index=["Small", "Medium", "Large", "All"])
        for rule in rule_names:
            for fraction in fractions:
                col_name = f"{rule} m/{fraction}"
                for size_class in df.index:
                    values = grouped_shortfall[size_class][fraction][rule]
                    df.loc[size_class, col_name] = stat_fn(values) if values else 0
        return df.astype(float)

    def build_confidence_interval_table():
        df = pd.DataFrame(index=["Small", "Medium", "Large", "All"])
        for rule in rule_names:
            for fraction in fractions:
                col_name = f"{rule} m/{fraction}"
                for size_class in df.index:
                    values = grouped_violations[size_class][fraction][rule]

                    # Clean up: remove NaN/inf and convert to numpy array
                    clean_values = np.array(values)
                    clean_values = clean_values[~np.isnan(clean_values)]
                    clean_values = clean_values[np.isfinite(clean_values)]

                    if len(clean_values) > 1:
                        mean = np.mean(clean_values)
                        sem = stats.sem(clean_values)

                        if np.isfinite(sem) and sem > 0:
                            ci = stats.t.interval(0.95, len(clean_values)-1, loc=mean, scale=sem)
                            df.loc[size_class, col_name] = f"[{ci[0]:.3f}, {ci[1]:.3f}]"
                        else:
                            df.loc[size_class, col_name] = "N/A"
                    else:
                        df.loc[size_class, col_name] = "N/A"
        return df

    avg_violations_df = build_violation_table(np.mean)
    max_violations_df = build_violation_table(np.max)
    avg_shortfall_df = build_shortfall_table(np.mean)
    ci_df = build_confidence_interval_table()

    def plot_violation_tables_split(df, title_prefix):
        fig, axes = plt.subplots(1, 5, figsize=(24, 4), constrained_layout=True)

        for i, (rule, color) in enumerate(colors.items()):
            cols = [col for col in df.columns if col.startswith(rule)]
            sub_df = df[cols]

            sns.heatmap(
                sub_df,
                annot=True,
                fmt=".4f",
                cmap=sns.light_palette(color, as_cmap=True),
                linewidths=0.5,
                cbar=False,
                ax=axes[i]
            )

            axes[i].set_title(rule, fontsize=12, color=color)
            axes[i].set_xlabel("Committee Size")
            axes[i].set_ylabel("File Size Category")
            axes[i].tick_params(axis='x', labelrotation=45)
            axes[i].set_yticklabels(sub_df.index, rotation=0, fontsize=10)

        fig.suptitle(f"{title_prefix} EJR+ Violations by Rule and Committee Size", fontsize=16)
        plt.show()

    def plot_ci_tables_split(df, title_prefix):
        fig, axes = plt.subplots(1, 5, figsize=(32, 4), constrained_layout=True)

        for i, (rule, color) in enumerate(colors.items()):
            cols = [col for col in df.columns if col.startswith(rule)]
            sub_df = df[cols]

            # Create a simple table visualization for CI
            for row_idx, (row_name, row_data) in enumerate(sub_df.iterrows()):
                for col_idx, value in enumerate(row_data):
                    axes[i].text(col_idx + 0.5, len(sub_df) - row_idx - 0.5, str(value),
                               ha='center', va='center', fontsize=8)

            axes[i].set_xlim(0, len(cols))
            axes[i].set_ylim(0, len(sub_df))
            axes[i].set_xticks(range(len(cols)))
            axes[i].set_xticklabels([col.split(' ', 1)[1] for col in cols], rotation=45)
            axes[i].set_yticks(range(len(sub_df)))
            axes[i].set_yticklabels(reversed(sub_df.index))
            axes[i].set_title(rule, fontsize=12, color=color)
            axes[i].set_xlabel("Committee Size")
            axes[i].set_ylabel("File Size Category")
            axes[i].grid(True, alpha=0.3)

        fig.suptitle(f"{title_prefix} 95% Confidence Intervals by Rule and Committee Size", fontsize=16)
        plt.show()

    plot_violation_tables_split(avg_violations_df, "Average")
    plot_violation_tables_split(max_violations_df, "Max")
    plot_violation_tables_split(avg_shortfall_df, "Average Shortfall")
    plot_ci_tables_split(ci_df, "Violations")

    # Print average run time for each algorithm
    logging.info(f"\n{Fore.YELLOW}=== Average running times ==={Style.RESET_ALL}")
    for algo, data in timing_data.items():
        avg_time = data["total_time"] / data["count"]
        logging.info(f"{Fore.LIGHTYELLOW_EX}{algo}: {Fore.WHITE}{avg_time:.4f} seconds{Style.RESET_ALL}")

    logging.info(f"\n{Fore.YELLOW}=== File Counts by Size Category ==={Style.RESET_ALL}")
    for size_class in ["Small", "Medium", "Large", "All"]:
        logging.info(f"{Fore.LIGHTYELLOW_EX}{size_class}: {Fore.WHITE}{file_size_counter[size_class]} files {Style.RESET_ALL}")
        for pair in file_metadata[size_class]:
            logging.info(f"{Fore.BLACK}   (m={pair[0]}, n={pair[1]}){Style.RESET_ALL}")

# Experiment 2

In [ ]:
def parse_toi_file(path, max_utility=5, num_candidates=100):
    """
    Parses .toi formatted preference files, preserving ties.
    
    In .toi files, braces {a, b} indicate that candidates a and b are tied 
    at the same rank.
    """
    with open(path, 'r') as f:
        # Filter out empty lines and comments
        data_lines = [line.strip() for line in f if line.strip() and not line.startswith('#')]
    
    utilities = np.zeros((len(data_lines), num_candidates), dtype=int)

    for i, line in enumerate(data_lines):
        # Remove the count at the start (e.g., "1: {1,2},3" -> "{1,2},3")
        line_content = line.split(':', 1)[-1].strip()
        
        # --- Parse the line into ranked groups ---
        ranked_groups = []
        current_token = ''
        in_braces = False
        
        for ch in line_content:
            if ch == '{':
                in_braces = True
                current_token += ch
            elif ch == '}':
                in_braces = False
                current_token += ch
            elif ch == ',' and not in_braces:
                ranked_groups.append(current_token.strip())
                current_token = ''
            else:
                current_token += ch
        if current_token:
            ranked_groups.append(current_token.strip())

        # --- Assign Utilities ---
        current_utility = max_utility
        for token in ranked_groups:
            # Check if this token represents a tie (e.g., "{1, 2}")
            if token.startswith('{') and token.endswith('}'):
                # Clean braces and split
                group_ids = [int(x) - 1 for x in token.strip('{}').split(',')]
                for idx in group_ids:
                    if 0 <= idx < num_candidates:
                        utilities[i][idx] = current_utility
            else:
                # Single candidate
                if token:
                    idx = int(token) - 1
                    if 0 <= idx < num_candidates:
                        utilities[i][idx] = current_utility
            
            # Decrease utility only once per group (preserving ties)
            current_utility -= 1

    return utilities

# --- Parse the csv file and convert to utility matrix ---
def parse_ratings_file(path, dataset_name=None):
    num_users = 1000 if dataset_name == "100K_Dataset" else 0
    num_movies = 1700 if dataset_name == "100K_Dataset" else 0

    utilities = np.zeros((num_users, num_movies), dtype=float)

    def split_line(line, delimiter='\t'):
        return line.strip().split(delimiter)

    with open(path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue  # Skip empty lines

            delimiter = '\t' if dataset_name == "100K_Dataset" else None

            if delimiter is None:
                logging.warning("Invalid Dataset Name!")
                return None

            userId_str, movieId_str, rating_str, timestamp = split_line(line, delimiter)

            userId = int(userId_str) - 1
            movieId = int(movieId_str) - 1
            rating = float(rating_str)

            utilities[userId][movieId] = rating

    return utilities

def average_satisfaction_sushi(utils, winners):
    num_candidates = utils.shape[1]
    valid_winners = [w for w in winners if 0 <= w < num_candidates]

    if not valid_winners:
        return 0.0

    satisfaction = np.sum(utils[:, valid_winners], axis=1)
    return np.mean(satisfaction)


def exclusion_ratio_sushi(utils, winners):
    num_candidates = utils.shape[1]
    valid_winners = [w for w in winners if 0 <= w < num_candidates]

    if not valid_winners:
        return 1.0

    covered = (utils[:, valid_winners] > 0).any(axis=1)
    ratio = 1.0 - np.mean(covered)
    return ratio


def percentile_utility_sushi(utils, winners, percentile):
    num_candidates = utils.shape[1]
    valid_winners = [w for w in winners if 0 <= w < num_candidates]

    if not valid_winners:
        return 0.0

    utils_of_winners = np.sum(utils[:, valid_winners], axis=1)
    return np.percentile(utils_of_winners, percentile)

def gini_coefficient_sushi(x):
    x = np.array(x, dtype=np.float64)
    if x.size == 0:
        return float('nan')
    if np.all(x == 0):
        return 0.0
    x = np.sort(x)
    n = x.size
    cumx = np.cumsum(x)
    gini = (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n
    return gini


def compare_algorithms_on_metrics(data_file_path, k_values, seeds, data_type=None, dataset_name=None):
    utils = (
        parse_toi_file(data_file_path) if data_type == "sushi"
        else parse_ratings_file(data_file_path, dataset_name) if data_type == "movies"
        else None
    )

    if utils is None:
        logging.warning("Invalid Data Type!")
        return

    num_candidates = utils.shape[1]

    project_list, voter_list, votes_by_voter = convert_matrix_utils(utils, "SubmodularSecretary")

    alg_names = ["Greedy Budgeting", "Online MES", "Online BOS", "Online Nash Rule"]

    best_counts = defaultdict(lambda: defaultdict(int))
    top2_counts = defaultdict(lambda: defaultdict(int))
    worst_counts = defaultdict(lambda: defaultdict(int))

    second_counts = defaultdict(lambda: defaultdict(int))
    third_counts = defaultdict(lambda: defaultdict(int))
    fourth_counts = defaultdict(lambda: defaultdict(int))

    best_counts_all = defaultdict(lambda: defaultdict(int))
    top2_counts_all = defaultdict(lambda: defaultdict(int))
    worst_counts_all = defaultdict(lambda: defaultdict(int))

    second_counts_all = defaultdict(lambda: defaultdict(int))
    third_counts_all = defaultdict(lambda: defaultdict(int))
    fourth_counts_all = defaultdict(lambda: defaultdict(int))

    total_runs_per_k = {k: 0 for k in k_values}
    total_runs_all = 0

    sum_metrics_all = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
    count_metrics_all = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))


    for k in k_values:
        readable_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        logging.info(f"{Fore.BLACK}Start time: {readable_time} -------> K Value = {k}{Style.RESET_ALL}")

        for seed in seeds:
            total_runs_per_k[k] += 1
            total_runs_all += 1

            results = {}

            try:
                results["Greedy Budgeting"] = greedy(utils, k, seed)
                results["Online MES"] = online_mes(utils, k, seed, 'utilitarian_greedy')
                results["Online BOS"] = online_bos(utils, k, seed)
                results["Online Nash Rule"] = monotone_submodular_secretary(
                    k,
                    projects=project_list,
                    voters=voter_list,
                    votes=votes_by_voter,
                    vote_type="Additive",
                    seed=seed
                )

            except Exception as e:
                logging.error(f"Algorithm run error for k={k}, seed={seed}: {e}")
                continue

            metrics_per_alg = {
                "average_satisfaction": {},
                "exclusion_ratio": {},
                "percentile_10": {},
                "percentile_15": {},
                "percentile_25": {},
                "gini_coefficient": {}
            }

            for alg in alg_names:
                winners = results[alg]


                try:
                    avg_sat = average_satisfaction_sushi(utils, winners)
                    excl_ratio = exclusion_ratio_sushi(utils, winners)
                    p10 = percentile_utility_sushi(utils, winners, 10)
                    p15 = percentile_utility_sushi(utils, winners, 15)
                    p25 = percentile_utility_sushi(utils, winners, 25)
                    utils_of_winners = np.sum(utils[:, winners], axis=1)
                    gini = gini_coefficient_sushi(utils_of_winners)
                except Exception as e:
                    logging.error(f"Metric calculation error for {alg}, k={k}, seed={seed}: {e}")
                    avg_sat = excl_ratio = p10 = p15 = p25 = gini = float('nan')

                metrics_per_alg["average_satisfaction"][alg] = avg_sat
                metrics_per_alg["exclusion_ratio"][alg] = excl_ratio
                metrics_per_alg["percentile_10"][alg] = p10
                metrics_per_alg["percentile_15"][alg] = p15
                metrics_per_alg["percentile_25"][alg] = p25
                metrics_per_alg["gini_coefficient"][alg] = gini

                # === Added: accumulate sums for average reporting ===
                for metric, value in [
                    ("average_satisfaction", avg_sat),
                    ("exclusion_ratio", excl_ratio),
                    ("percentile_10", p10),
                    ("percentile_15", p15),
                    ("percentile_25", p25),
                    ("gini_coefficient", gini),
                ]:
                    if not np.isnan(value):
                        sum_metrics_all[metric][k][alg] += value
                        count_metrics_all[metric][k][alg] += 1


            for metric, values in metrics_per_alg.items():
                higher_is_better = metric not in ("exclusion_ratio", "gini_coefficient")

                sorted_algs = sorted(
                    values.items(),
                    key=lambda x: x[1],
                    reverse=higher_is_better
                )

                ordered_algs = [alg for alg, val in sorted_algs if not np.isnan(val)]

                if not ordered_algs:
                    continue

                best_alg = ordered_algs[0]
                best_counts[metric][(k, best_alg)] += 1
                best_counts_all[metric][best_alg] += 1

                for top_alg in ordered_algs[:2]:
                    top2_counts[metric][(k, top_alg)] += 1
                    top2_counts_all[metric][top_alg] += 1

                worst_alg = ordered_algs[-1]
                worst_counts[metric][(k, worst_alg)] += 1
                worst_counts_all[metric][worst_alg] += 1

                if len(ordered_algs) > 1:
                    second_alg = ordered_algs[1]
                    second_counts[metric][(k, second_alg)] += 1
                    second_counts_all[metric][second_alg] += 1

                if len(ordered_algs) > 2:
                    third_alg = ordered_algs[2]
                    third_counts[metric][(k, third_alg)] += 1
                    third_counts_all[metric][third_alg] += 1

                if len(ordered_algs) > 3:
                    fourth_alg = ordered_algs[3]
                    fourth_counts[metric][(k, fourth_alg)] += 1
                    fourth_counts_all[metric][fourth_alg] += 1

    logging.info(f"\n{Fore.YELLOW}=== Overall Results based on size of k ==={Style.RESET_ALL}\n")
    size_categories = {
        "Small k (2, 5, 8)": [2, 5, 8],
        "Medium k (15, 20, 25)": [15, 20, 25],
        "Large k (35, 40, 45)": [35, 40, 45],
    }
    for category_name, k_list in size_categories.items():
        logging.info(f"{Fore.LIGHTYELLOW_EX}{category_name}{Style.RESET_ALL}")
        for metric in ["average_satisfaction", "exclusion_ratio", "percentile_10", "percentile_15", "percentile_25", "gini_coefficient"]:
            logging.info(f" Metric: {metric}")
            total_runs_category = sum(total_runs_per_k[k] for k in k_list if k in total_runs_per_k) or 1
            for alg in alg_names:
                best_sum = sum(best_counts[metric][(k, alg)] for k in k_list if (k, alg) in best_counts[metric])
                second_sum = sum(second_counts[metric][(k, alg)] for k in k_list if (k, alg) in second_counts[metric])
                third_sum = sum(third_counts[metric][(k, alg)] for k in k_list if (k, alg) in third_counts[metric])
                fourth_sum = sum(fourth_counts[metric][(k, alg)] for k in k_list if (k, alg) in fourth_counts[metric])
                top2_sum = sum(top2_counts[metric][(k, alg)] for k in k_list if (k, alg) in top2_counts[metric])
                worst_sum = sum(worst_counts[metric][(k, alg)] for k in k_list if (k, alg) in worst_counts[metric])

                best_pct = 100 * best_sum / total_runs_category
                second_pct = 100 * second_sum / total_runs_category
                third_pct = 100 * third_sum / total_runs_category
                fourth_pct = 100 * fourth_sum / total_runs_category
                top2_pct = 100 * top2_sum / total_runs_category
                worst_pct = 100 * worst_sum / total_runs_category

                logging.info(
                    f"  {alg:20s} {Fore.BLACK}Best:{Style.RESET_ALL} {best_pct:6.2f}%, {Fore.BLACK}2nd:{Style.RESET_ALL} {second_pct:6.2f}%, "
                    f"{Fore.BLACK}3rd:{Style.RESET_ALL} {third_pct:6.2f}%, {Fore.BLACK}4th:{Style.RESET_ALL} {fourth_pct:6.2f}%, "
                    f"{Fore.BLACK}Top2:{Style.RESET_ALL} {top2_pct:6.2f}%, {Fore.BLACK}Worst:{Style.RESET_ALL} {worst_pct:6.2f}%"
                )

    logging.info(f"\n{Fore.YELLOW}=== Overall Results (all k combined) ==={Style.RESET_ALL}\n")
    for metric in ["average_satisfaction", "exclusion_ratio", "percentile_10", "percentile_15", "percentile_25", "gini_coefficient"]:
        logging.info(f"{Fore.LIGHTYELLOW_EX}Metric: {metric}{Style.RESET_ALL}")
        denom = total_runs_all or 1
        for alg in alg_names:
            best_pct = 100 * best_counts_all[metric][alg] / denom
            second_pct = 100 * second_counts_all[metric][alg] / denom
            third_pct = 100 * third_counts_all[metric][alg] / denom
            fourth_pct = 100 * fourth_counts_all[metric][alg] / denom
            top2_pct = 100 * top2_counts_all[metric][alg] / denom
            worst_pct = 100 * worst_counts_all[metric][alg] / denom
            logging.info(
                    f"  {alg:20s} {Fore.BLACK}Best:{Style.RESET_ALL} {best_pct:6.2f}%, {Fore.BLACK}2nd:{Style.RESET_ALL} {second_pct:6.2f}%, "
                    f"{Fore.BLACK}3rd:{Style.RESET_ALL} {third_pct:6.2f}%, {Fore.BLACK}4th:{Style.RESET_ALL} {fourth_pct:6.2f}%, "
                    f"{Fore.BLACK}Top2:{Style.RESET_ALL} {top2_pct:6.2f}%, {Fore.BLACK}Worst:{Style.RESET_ALL} {worst_pct:6.2f}%"
                )

    logging.info(f"\n{Fore.YELLOW}=== Average Metric Values per Algorithm by k Size Category ==={Style.RESET_ALL}\n")
    for category_name, k_list in size_categories.items():
        logging.info(f"{Fore.LIGHTYELLOW_EX}{category_name}{Style.RESET_ALL}")
        for metric in ["average_satisfaction", "exclusion_ratio", "percentile_10", "percentile_15", "percentile_25", "gini_coefficient"]:
            logging.info(f" Metric: {metric}")
            for alg in alg_names:
                total_val = 0.0
                total_count = 0
                for k in k_list:
                    total_val += sum_metrics_all[metric][k][alg]
                    total_count += count_metrics_all[metric][k][alg]
                avg_val = total_val / total_count if total_count > 0 else float('nan')
                logging.info(f"  {alg:20s} {Fore.BLACK}Average Value:{Style.RESET_ALL} {avg_val:.4f}")

    logging.info(f"\n{Fore.YELLOW}=== Average Metric Values (all k combined) ==={Style.RESET_ALL}\n")
    for metric in ["average_satisfaction", "exclusion_ratio", "percentile_10", "percentile_15", "percentile_25", "gini_coefficient"]:
        logging.info(f"{Fore.LIGHTYELLOW_EX}Metric: {metric}{Style.RESET_ALL}")
        for alg in alg_names:
            total_val = 0.0
            total_count = 0
            for k in k_values:
                total_val += sum_metrics_all[metric][k][alg]
                total_count += count_metrics_all[metric][k][alg]
            avg_val = total_val / total_count if total_count > 0 else float('nan')
            logging.info(f"  {alg:20s} {Fore.BLACK}Average Value:{Style.RESET_ALL} {avg_val:.4f}")

def run_experiment_two_A():
    k_values = [2, 5, 8, 15, 20, 25, 35, 40, 45]
    seeds = list(range(20))
    toi_file_path = "sample_data/Sushi.txt"
    compare_algorithms_on_metrics(toi_file_path, k_values, seeds, data_type='sushi', dataset_name="Sushi")

def run_experiment_two_B():
    k_values = [2, 5, 8, 15, 20, 25, 35, 40, 45]
    seeds = list(range(50))
    data_file_path = "sample_data/MoviesRatings100k.csv"
    compare_algorithms_on_metrics(data_file_path, k_values=k_values, seeds=seeds, data_type='movies', dataset_name="100K_Dataset")

# Experiment 3.

In [ ]:
def create_random_utilities(n, m, positive_fraction, max_utility, seed=None):
    """Generates Impartial Culture utilities (Random Approval)."""
    if seed is not None:
        np.random.seed(seed)
    utilities = np.zeros((n, m), dtype=int)
    for i in range(n):
        num_positive = np.random.binomial(m, positive_fraction)
        if num_positive == 0: continue
        positive_candidates = np.random.choice(m, num_positive, replace=False)
        mean_utility = 0.75 * max_utility
        std_dev = 0.7 * max_utility
        utilities[i, positive_candidates] = np.clip(
            np.random.normal(loc=mean_utility, scale=std_dev, size=num_positive).round(), 
            1, max_utility
        ).astype(int)
    return utilities

### Part A

In [ ]:
def plot_metric_normalized_exp3_A(metric):
    output_dir = "./plots"
    os.makedirs(output_dir, exist_ok=True)

    plt.rcParams.update({
        'font.family': 'Times New Roman',  # Set font to Times New Roman
        'axes.titlesize': 20,     # Title font size
        'axes.labelsize': 18,     # X and Y axis label size
        'xtick.labelsize': 16,    # X-axis tick label size
        'ytick.labelsize': 16,    # Y-axis tick label size
        'legend.fontsize': 16,    # Legend font size
    })

    var_labels = {
        "n": "Number of Voters",
        "m": "Number of Candidates",
        "k": "Committee Size",
        "p": "Approval Probability"
    }

    for var in ["n", "m", "k", "p"]:
        plt.figure(figsize=(12, 6))
        plt.title(f"{metric} normalized to Offline MES (varied by {var_labels[var]})")

        x_vals = sorted(set(entry[var] for entry in all_results))
        offline_mes_baseline = {}

        for v in x_vals:
            entries = [entry for entry in all_results if entry[var] == v and entry["algorithm"] == "Offline MES"]
            if entries:
                baseline_value = np.mean([entry[metric] for entry in entries])
                offline_mes_baseline[v] = baseline_value

        for alg in color_marker:
            color, marker = color_marker[alg]
            clean_label = alg.replace(" with utilitarian completion", "")
            if clean_label == "Greedy":
                clean_label = "Greedy Budgeting"

            y_vals, y_errs, x_vals_filtered = [], [], []

            for v in x_vals:
                entries = [entry for entry in all_results if entry[var] == v and entry["algorithm"] == alg]
                if not entries or v not in offline_mes_baseline:
                    continue

                y = np.mean([entry[metric] for entry in entries])
                err = np.mean([entry[metric + " std"] for entry in entries])
                baseline = offline_mes_baseline[v]

                if baseline > 0:
                    normalized_y = y / baseline
                    normalized_err = err / baseline
                else:
                    normalized_y = 0
                    normalized_err = 0

                y_vals.append(normalized_y)
                y_errs.append(normalized_err)
                x_vals_filtered.append(v)

            if y_vals:
                plt.errorbar(x_vals_filtered, y_vals, yerr=y_errs, label=clean_label,
                             color=color, marker=marker, linestyle='-')

        plt.xlabel(var_labels[var])
        plt.ylabel(f"{metric} (relative to Offline MES)")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()

        # Save and show the plot
        check_save = True
        check_save_counter = 0
        while check_save:
            plot_filename = f"{output_dir}/{metric.replace(' ', '_')}_normalized_{var}_{check_save_counter}.png"
            check_save_counter += 1

            if not os.path.exists(plot_filename):
                plt.savefig(plot_filename)
                check_save = False
                break

        plt.show()
        plt.close()

    return "All plots generated and saved successfully."

In [ ]:
# Experiment settings
n_values = [5, 10, 20, 30, 50, 70, 100, 150]
m_k_pairs = [
    (10, 2), (10, 4),
    (20, 2), (20, 4), (20, 5),
    (30, 2), (30,4), (30, 5), (30, 10),
    (50, 5), (50, 10), (50, 12), (50, 15),
    (80, 10), (80, 12), (80, 15), (80, 20), (80, 30),
    (100, 12), (100, 15), (100, 20), (100, 30),
    (120, 15), (120, 20), (120,30), (120, 40),
    (150, 20), (150, 30), (150, 40),
    (200, 20), (200, 30), (200, 40)]

p_values = [0.2, 0.4, 0.6, 0.8, 1]
max_utility = 200
seed_values = range(0,10)
all_results = []

def run_experiment_three_A():
    for n in n_values:
        # Monitor the state of the ongoing process
        readable_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        logging.info(f"{Fore.BLACK}Start time: {readable_time} -------> N Value = {n}{Style.RESET_ALL}")
        for (m, k), p in itertools.product(m_k_pairs, p_values):
            for seed in seed_values:
                utils = create_random_utilities(n, m, p, max_utility, seed)

                result = compare_algorithms([k], [seed], n, m, p, max_utility, utils)

                for alg in result[k]:
                    metrics = result[k][alg]
                    all_results.append({
                        "n": n, "m": m, "k": k, "p": p, "seed": seed, "algorithm": alg,
                        "Avg Satisfaction": np.mean(metrics["Avg Satisfaction"]),
                        "Avg Satisfaction std": np.std(metrics["Avg Satisfaction"]),
                        "Exclusion Ratio": np.mean(metrics["Exclusion Ratio"]),
                        "Exclusion Ratio std": np.std(metrics["Exclusion Ratio"]),
                        "Gini": np.mean(metrics["Gini"]),
                        "Gini std": np.std(metrics["Gini"]),
                        "10th Percentile": np.mean(metrics["10th Percentile"]),
                        "10th Percentile std": np.std(metrics["10th Percentile"]),
                        "15th Percentile": np.mean(metrics["15th Percentile"]),
                        "15th Percentile std": np.std(metrics["15th Percentile"]),
                        "25th Percentile": np.mean(metrics["25th Percentile"]),
                        "25th Percentile std": np.std(metrics["25th Percentile"]),
                        "m/k": m / k,
                        "k/n": k / n
                    })


    for metric in ["Avg Satisfaction", "Exclusion Ratio", "Gini", "10th Percentile", "15th Percentile", "25th Percentile"]:
        plot_metric(metric)
        plot_relative_to_mes(metric)
        plot_metric_normalized_exp3_A(metric)

### Part B

In [ ]:
def plot_metric_normalized_exp3_BandC(metric):
    output_dir = "./plots"
    os.makedirs(output_dir, exist_ok=True)

    plt.rcParams.update({
        'font.family': 'Times New Roman',  # Set font to Times New Roman
        'axes.titlesize': 20,     # Title font size
        'axes.labelsize': 18,     # X and Y axis label size
        'xtick.labelsize': 16,    # X-axis tick label size
        'ytick.labelsize': 16,    # Y-axis tick label size
        'legend.fontsize': 16,    # Legend font size
    })

    var_labels = {
        "n": "Number of Voters",
        "m": "Number of Candidates",
        "k": "Committee Size",
        "phi": "Dispersion Parameter (ϕ)"
    }

    for var in ["n", "m", "k", "phi"]:
        plt.figure(figsize=(12, 6))
        plt.title(f"{metric} (Normalized to Offline MES) vs {var_labels[var]}")

        x_vals = sorted(set(entry[var] for entry in all_results))
        offline_mes_baseline = {}

        for v in x_vals:
            entries = [entry for entry in all_results if entry[var] == v and entry["algorithm"] == "Offline MES"]
            if entries:
                baseline_value = np.mean([entry[metric] for entry in entries])
                offline_mes_baseline[v] = baseline_value

        for alg in color_marker:
            color, marker = color_marker[alg]
            clean_label = alg.replace(" with utilitarian completion", "")
            if clean_label == "Greedy":
                clean_label = "Greedy Budgeting"

            y_vals, y_errs, x_vals_filtered = [], [], []

            for v in x_vals:
                entries = [entry for entry in all_results if entry[var] == v and entry["algorithm"] == alg]
                if not entries or v not in offline_mes_baseline:
                    continue

                y = np.mean([entry[metric] for entry in entries])
                err = np.mean([entry[metric + " std"] for entry in entries])

                baseline = offline_mes_baseline[v]
                if baseline > 0:  # Avoid division by zero
                    normalized_y = y / baseline
                    normalized_err = err / baseline
                else:
                    normalized_y = 0
                    normalized_err = 0

                y_vals.append(normalized_y)
                y_errs.append(normalized_err)
                x_vals_filtered.append(v)

            if y_vals:
                plt.errorbar(x_vals_filtered, y_vals, yerr=y_errs, label=clean_label,
                             color=color, marker=marker, linestyle='-')

        plt.xlabel(var_labels[var])
        plt.ylabel(f"{metric} (relative to Offline MES)")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()

        # Save and show the plot
        check_save = True
        check_save_counter = 0
        while check_save:
            plot_filename = f"{output_dir}/{metric.replace(' ', '_')}_normalized_vs_{var}_{check_save_counter}.png"
            check_save_counter += 1
            if not os.path.exists(plot_filename):
                plt.savefig(plot_filename)
                check_save = False
                break

        plt.show()
        plt.close()

    return "All normalized plots saved in './plots'."


In [ ]:
def sample_mallows_ranking(reference, phi):
    """Samples one ranking from the Mallows model using the insert-based method."""
    ranking = [reference[0]]
    for i in range(1, len(reference)):
        probs = [phi ** j for j in range(i + 1)]
        probs = np.array(probs) / sum(probs)
        insert_pos = np.random.choice(i + 1, p=probs)
        ranking.insert(insert_pos, reference[i])
    return ranking

def create_mallows_utilities(n, m, phi=0.8, max_utility=10, min_utility=1,
                            utility_noise=2, seed=None, return_rankings=False):

    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    reference_ranking = list(range(m))
    utilities = np.zeros((n, m), dtype=int)
    rankings = []

    for i in range(n):
        ranking = sample_mallows_ranking(reference_ranking, phi)
        rankings.append(ranking)


        base_range = max_utility - min_utility
        step = base_range / (m - 1) if m > 1 else base_range

        base_utilities = np.zeros(m, dtype=int)
        for j, candidate in enumerate(ranking):
            base_utilities[candidate] = max(min_utility, int(max_utility - j * step))

        for j in range(m):
            candidate = ranking[j]

            if j == 0:
                if j + 1 < m:
                    lower_bound = base_utilities[ranking[j+1]] + 1
                else:
                    lower_bound = min_utility
                upper_bound = max_utility
            elif j == m - 1:
                lower_bound = min_utility
                upper_bound = base_utilities[ranking[j-1]] - 1
            else:
                lower_bound = base_utilities[ranking[j+1]] + 1
                upper_bound = base_utilities[ranking[j-1]] - 1

            # Apply noise within safe bounds
            noise_range = min(utility_noise, (upper_bound - lower_bound) // 2)
            if noise_range > 0:
                noise = np.random.randint(-noise_range, noise_range + 1)
                utilities[i, candidate] = max(lower_bound, min(upper_bound, base_utilities[candidate] + noise))
            else:
                utilities[i, candidate] = base_utilities[candidate]

        for j in range(m-1):
            if utilities[i, ranking[j]] <= utilities[i, ranking[j+1]]:
                utilities[i, ranking[j]] = utilities[i, ranking[j+1]] + 1

    if return_rankings:
        return utilities, rankings
    return utilities



In [ ]:
def group_by_param_and_algorithm(results, param_name):
    param_values = sorted(list(set(result[param_name] for result in results if param_name in result)))
    alg_names = sorted(list(set(result["algorithm"] for result in results)))

    grouped_data = {
        param_val: {alg: [] for alg in alg_names}
        for param_val in param_values
    }

    for result in results:
        param_val = result[param_name]
        alg = result["algorithm"]
        grouped_data[param_val][alg].append(result)

    return grouped_data, param_values, alg_names

def plot_metric_by_param(metric, param_name, xlabel, results=None):


    if results is None:
        results = all_results

    color_marker = {
        "Online BOS": ("#2ca02c", "o"),
        "Offline MES": ("#000000", "x"),
        "Greedy Budgeting": ("#1f77b4", "s"),
        "Online MES": ("#ff7f0e", "^"),
        "Online Nash Rule": ("#d62728", "+")
    }

    param_labels = {
        "n": "Number of Voters",
        "m": "Number of Candidates",
        "k": "Committee Size",
        "phi": "Dispersion Parameter (ϕ)",
        "m/k": "Candidates per Committee Member (m/k)",
        "k/n": "Committee Size as Fraction of Voters (k/n)"
    }

    readable_xlabel = param_labels.get(param_name, xlabel)

    filtered_results = [r for r in results if not np.isnan(r[metric])]

    grouped_data, param_values, alg_names = group_by_param_and_algorithm(filtered_results, param_name)

    plt.rcParams.update({
        'font.family': 'Times New Roman',  # Set font to Times New Roman
        'axes.titlesize': 20,     # Title font size
        'axes.labelsize': 18,     # X and Y axis label size
        'xtick.labelsize': 16,    # X-axis tick label size
        'ytick.labelsize': 16,    # Y-axis tick label size
        'legend.fontsize': 16,    # Legend font size
    })


    plt.figure(figsize=(12, 6))

    for alg in alg_names:
        x_values = []
        y_values = []
        yerr_values = []

        for param_val in param_values:
            alg_results = grouped_data[param_val][alg]
            if alg_results:
                x_values.append(param_val)
                y_values.append(np.mean([r[metric] for r in alg_results]))
                yerr_values.append(np.std([r[metric] for r in alg_results]) / np.sqrt(len(alg_results)))

        if x_values:
            color, marker = color_marker.get(alg, ("gray", "o"))  # Fallback to gray if not found
            plt.errorbar(x_values, y_values, yerr=yerr_values, label=alg,
                         color=color, marker=marker, linestyle='-', capsize=5)

    plt.xlabel(readable_xlabel)
    plt.ylabel(metric)
    plt.title(f"{metric} vs {readable_xlabel}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()





def plot_metrics_against_parameters():
    metrics = ["Avg Satisfaction", "Exclusion Ratio", "Gini", "10th Percentile", "15th Percentile", "25th Percentile"]

    # Plot against n (number of voters)
    for metric in metrics:
        # Get results for each value of n, averaging over other parameters
        plot_metric_by_param(metric, "n", "Number of Voters (n)", all_results)

    # Plot against phi (dispersion parameter)
    for metric in metrics:
        plot_metric_by_param(metric, "phi", "Dispersion Parameter (phi)", all_results)

    for metric in metrics:
        plot_metric_normalized_exp3_BandC(metric)

In [ ]:
# Experiment settings
n_values = [5, 10, 20, 30, 50, 70, 100, 150]
m_k_pairs = [
    (10, 2), (10, 4),
    (20, 2), (20, 4), (20, 5),
    (30, 2), (30,4), (30, 5), (30, 10),
    (50, 5), (50, 10), (50, 12), (50, 15),
    (80, 10), (80, 12), (80, 15), (80, 20), (80, 30),
    (100, 12), (100, 15), (100, 20), (100, 30),
    (120, 15), (120, 20), (120,30), (120, 40),
    (150, 20), (150, 30), (150, 40),
    (200, 20), (200, 30), (200, 40)]

phi_values = [0.2, 0.4, 0.6, 0.8, 1]
max_utility = 200
min_utility = 0
utility_noise = 50
seed_values = range(0, 10)

all_results = []

def run_experiment_three_B():
    for n in n_values:
        # Monitor the state of the ongoing process
        readable_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        logging.info(f"{Fore.BLACK}Start time: {readable_time} -------> N Value = {n}{Style.RESET_ALL}")
        for (m, k), phi in itertools.product(m_k_pairs, phi_values):
            for seed in seed_values:
                utils = create_mallows_utilities(n, m, phi=phi, max_utility=max_utility, min_utility=min_utility,
                                                 utility_noise=utility_noise, seed=seed)

                result = compare_algorithms([k], [seed], n, m, phi, max_utility, utils)


                for alg in result[k]:
                    metrics = result[k][alg]
                    all_results.append({
                        "n": n, "m": m, "k": k, "phi": phi, "seed": seed, "algorithm": alg,
                        "Avg Satisfaction": np.mean(metrics["Avg Satisfaction"]),
                        "Avg Satisfaction std": np.std(metrics["Avg Satisfaction"]),
                        "Exclusion Ratio": np.mean(metrics["Exclusion Ratio"]),
                        "Exclusion Ratio std": np.std(metrics["Exclusion Ratio"]),
                        "Gini": np.mean(metrics["Gini"]),
                        "Gini std": np.std(metrics["Gini"]),
                        "10th Percentile": np.mean(metrics["10th Percentile"]),
                        "10th Percentile std": np.std(metrics["10th Percentile"]),
                        "15th Percentile": np.mean(metrics["15th Percentile"]),
                        "15th Percentile std": np.std(metrics["15th Percentile"]),
                        "25th Percentile": np.mean(metrics["25th Percentile"]),
                        "25th Percentile std": np.std(metrics["25th Percentile"]),
                        "m/k": m / k,
                        "k/n": k / n
                    })

    # Generate all plots
    plot_metrics_against_parameters()

### Part C

In [ ]:
def create_mallows_utilities_normalized(n, m, phi=0.8, max_utility=10, min_utility=1,
                            utility_noise=2, seed=None):

    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    # Generate rankings using prefsampling
    rankings = norm_mallows(n, m, phi, seed=seed)


    utilities = np.zeros((n, m), dtype=int)

    for i in range(n):
        ranking = rankings[i]


        base_range = max_utility - min_utility
        step = base_range / (m - 1) if m > 1 else base_range

        base_utilities = np.zeros(m, dtype=int)
        for j, candidate in enumerate(ranking):
            base_utilities[candidate] = max(min_utility, int(max_utility - j * step))

        for j in range(m):
            candidate = ranking[j]

            if j == 0:
                if j + 1 < m:
                    lower_bound = base_utilities[ranking[j+1]] + 1
                else:
                    lower_bound = min_utility
                upper_bound = max_utility
            elif j == m - 1:
                lower_bound = min_utility
                upper_bound = base_utilities[ranking[j-1]] - 1
            else:
                lower_bound = base_utilities[ranking[j+1]] + 1
                upper_bound = base_utilities[ranking[j-1]] - 1

            noise_range = min(utility_noise, (upper_bound - lower_bound) // 2)
            if noise_range > 0:
                noise = np.random.randint(-noise_range, noise_range + 1)
                utilities[i, candidate] = max(lower_bound, min(upper_bound, base_utilities[candidate] + noise))
            else:
                utilities[i, candidate] = base_utilities[candidate]

        for j in range(m-1):
            if utilities[i, ranking[j]] <= utilities[i, ranking[j+1]]:
                utilities[i, ranking[j]] = utilities[i, ranking[j+1]] + 1

    return utilities

In [ ]:
# Experiment settings
n_values = [5, 10, 20, 30, 50, 70, 100, 150]
m_k_pairs = [
    (10, 2), (10, 4),
    (20, 2), (20, 4), (20, 5),
    (30, 2), (30,4), (30, 5), (30, 10),
    (50, 5), (50, 10), (50, 12), (50, 15),
    (80, 10), (80, 12), (80, 15), (80, 20), (80, 30),
    (100, 12), (100, 15), (100, 20), (100, 30),
    (120, 15), (120, 20), (120,30), (120, 40),
    (150, 20), (150, 30), (150, 40),
    (200, 20), (200, 30), (200, 40)]

phi_values = [0.2, 0.4, 0.6, 0.8, 1]
max_utility = 200
min_utility = 0
utility_noise = 50
seed_values = range(0, 10)

all_results = []

def run_experiment_three_C():
    for n in n_values:
        readable_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        logging.info(f"{Fore.BLACK}Start time: {readable_time} -------> N Value = {n}{Style.RESET_ALL}")
        for (m, k), phi in itertools.product(m_k_pairs, phi_values):
            for seed in seed_values:
                utils = create_mallows_utilities_normalized(n, m, phi=phi, max_utility=max_utility, min_utility=min_utility,
                                                 utility_noise=utility_noise, seed=seed)

                result = compare_algorithms([k], [seed], n, m, phi, max_utility, utils)

                for alg in result[k]:
                    metrics = result[k][alg]
                    all_results.append({
                        "n": n, "m": m, "k": k, "phi": phi, "seed": seed, "algorithm": alg,
                        "Avg Satisfaction": np.mean(metrics["Avg Satisfaction"]),
                        "Avg Satisfaction std": np.std(metrics["Avg Satisfaction"]),
                        "Exclusion Ratio": np.mean(metrics["Exclusion Ratio"]),
                        "Exclusion Ratio std": np.std(metrics["Exclusion Ratio"]),
                        "Gini": np.mean(metrics["Gini"]),
                        "Gini std": np.std(metrics["Gini"]),
                        "10th Percentile": np.mean(metrics["10th Percentile"]),
                        "10th Percentile std": np.std(metrics["10th Percentile"]),
                        "15th Percentile": np.mean(metrics["15th Percentile"]),
                        "15th Percentile std": np.std(metrics["15th Percentile"]),
                        "25th Percentile": np.mean(metrics["25th Percentile"]),
                        "25th Percentile std": np.std(metrics["25th Percentile"]),
                        "m/k": m / k,
                        "k/n": k / n
                    })

    # Generate all plots
    plot_metrics_against_parameters()

# Experiment 4

In [ ]:
def create_polarized_instance(n_voters, m_candidates, x_fraction, coh2, instance_seed):
    """
    Creates a polarized society with two groups.
    Group 1: Size x * n, approves first half of candidates.
    Group 2: Remainder, approves a random subset of the second half (controlled by coh2).
    """
    original_state = random.getstate()
    random.seed(instance_seed)
    
    assert m_candidates % 2 == 0
    group_1_size = int(n_voters * x_fraction)
    group_2_size = n_voters - group_1_size
    
    utils = np.zeros((n_voters, m_candidates), dtype=int)
    
    # Group 1 preferences
    utils[:group_1_size, :m_candidates // 2] = 1
    
    # Group 2 preferences
    for i in range(group_2_size):
        num_approved = int(coh2 * (m_candidates // 2))
        approved = random.sample(range(m_candidates // 2, m_candidates), num_approved)
        for j in approved:
            utils[group_1_size + i, j] = 1
            
    random.setstate(original_state)
    return utils

In [ ]:
def run_experiment_four(num_random_profiles_to_test=300, max_num_seeds=10):
    seeds = range(0, max_num_seeds)
    num_random_profiles = num_random_profiles_to_test

    underperformance_counts = {
        'Greedy Budgeting': 0,
        'Online MES': 0,
        'Online BOS': 0,
        'Online Nash Rule': 0
    }
    underperformance_total_gap = {
        'Greedy Budgeting': 0,
        'Online MES': 0,
        'Online BOS': 0,
        'Online Nash Rule': 0
    }
    underperformance_max_gap = {
        'Greedy Budgeting': 0,
        'Online MES': 0,
        'Online BOS': 0,
        'Online Nash Rule': 0
    }

    deficit_per_instance = {
        'Greedy Budgeting': [],
        'Online MES': [],
        'Online BOS': [],
        'Online Nash Rule': []
    }

    for round_num in range(num_random_profiles):
        random.seed(round_num)
        np.random.seed(round_num)

        if round_num % 50 == 0:
            readable_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            logging.info(f"{Fore.BLACK}Start time: {readable_time} -------> Round Number = {round_num+1}{Style.RESET_ALL}")

        n = random.choice(range(5, 200))
        m = random.choice(range(16, 300, 2))
        k = random.choice(range(3, int(m/4)))
        x = round(random.uniform(0.1, 0.9), 2)
        coh2 = round(random.uniform(0.1, 0.9), 2)

        utils = create_polarized_instance(n, m, x, coh2, round_num)
        group1 = set(range(m // 2))
        avg_opt = math.floor((math.floor(int(n * x)) / n) * k)

        g1_mes = []
        g1_bos = []
        g1_greedy = []
        g1_submod = []

        for seed in seeds:
            mes_sel = online_mes(utils, k, seed, 'utilitarian_greedy')
            bos_sel = online_bos(utils, k, seed)
            greedy_sel = greedy(utils, k, seed)
            projects_list, voters_list, votes = convert_matrix_utils(utils, "SubmodularSecretary")
            submod_sel = monotone_submodular_secretary(k, projects_list, voters_list, votes, vote_type="Approval", seed=seed)

            g1_mes.append(sum(1 for c in mes_sel if c in group1))
            g1_bos.append(sum(1 for c in bos_sel if c in group1))
            g1_greedy.append(sum(1 for c in greedy_sel if c in group1))
            g1_submod.append(sum(1 for c in submod_sel if c in group1))
        for name, counts in [('Online MES', g1_mes), ('Online BOS', g1_bos), ('Greedy Budgeting', g1_greedy), ('Online Nash Rule', g1_submod)]:
            avg_val = np.mean(counts)
            if avg_val < avg_opt:
                underperformance_counts[name] += 1
                deficit = avg_opt - avg_val
                underperformance_total_gap[name] += deficit
                deficit_per_instance[name].append(deficit)
                underperformance_max_gap[name] = max(underperformance_max_gap[name], deficit)

    underperformance_percentage = {
        method: (underperformance_counts[method] / num_random_profiles) * 100
        for method in underperformance_counts
    }

    avg_deficit_per_instance = {
        method: (underperformance_total_gap[method] / underperformance_counts[method])
        if underperformance_counts[method] > 0 else 0
        for method in underperformance_counts
    }

    plt.figure(figsize=(10, 6))
    methods = list(underperformance_counts.keys())
    percentages = [underperformance_percentage[m] for m in methods]
    plt.bar(methods, percentages, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#6C3BAA'])
    plt.title("Percentage of Instances Below avg_opt")
    plt.ylabel("Percentage of Total Profiles (%)")
    plt.grid(axis='y')
    plt.show()

    logging.info(f"{Fore.BLUE}Percentage of underperformance:")
    for method, percentage in zip(methods, percentages):
        logging.info(f"{Fore.YELLOW}{method}: {Fore.WHITE}{percentage:.2f}%")

    max_deficits = [underperformance_max_gap[m] for m in methods]
    avg_deficits = [avg_deficit_per_instance[m] for m in methods]

    x = np.arange(len(methods))
    width = 0.35  # Bar width

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, avg_deficits, width, label='Average Deficit', color='#ffdb0e')
    rects2 = ax.bar(x + width/2, max_deficits, width, label='Maximum Deficit', color='#8c3b04')

    ax.set_ylabel('Deficit')
    ax.set_title('Avg and Max Deficit per Instance Below avg_opt')
    ax.set_xticks(x)
    ax.set_xticklabels(methods)
    ax.legend()

    plt.grid(axis='y')
    plt.show()

    logging.info(f"{Fore.YELLOW}Average {Fore.WHITE}and {Fore.RED}Maximum deficits:")
    for m, avg, mx in zip(methods, avg_deficits, max_deficits):
        logging.info(f"{Fore.WHITE}{m} - {Fore.YELLOW}Avg: {avg:.4f}  {Fore.RED}Max: {mx:.4f}")